## OpenAI

In [3]:
import openai


def get_oai_response(prompt, model, functions, msgs):
  openai.api_key = "sk-"
  # openai.base_url = "http://localhost:8019/v1/"
  
  try:
    completion = openai.chat.completions.create(
      model=model,
      temperature=0.1,
      messages=msgs,
      tools=functions,
      tool_choice="auto",
      # functions=functions,
      # function_call="auto",
      stream=False,
    )
    return completion.choices[0]
  except Exception as e:
    print(e, model, prompt)


In [4]:
system_prompt = "You are a helpful assistant."
functions = [
   {
      "type": "function",
    "function": {
      "name": "getCurrentWeather",
      "description": "Get the weather in location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
          "unit": {"type": "string", "enum": ["c", "f"]}
        },
        "required": ["location"]
      }
    }
  },
   {    "type": "function",
     "function":
    {
        "name": "orderUmbrella",
        "description": "Do this to help user to order an umbrella online", 
        "parameters": {
            "type": "object",
            "properties": {
                "brand_name": {
                    "type": "string",
                    "description": "The name of the umbrella brand"
                }
            },
            "required": [
                "brand_name"
            ]
        }
    }},
]


user_query = "check the weather in boston"
# msgs = [{"role": "system", "content":system_prompt} ,{"role": "user", "content": user_query}, {"role": "function", "content": '<<functions>>[getCurrentWeather(location="Boston)]'}, {"role": "observation", "content": "<<observation>>72 f, rainy."}
#         ,
#         {"role": "assistant", "content": "The current weather in Boston is 72 degrees Fahrenheit and it's raining. Would you like to order an umbrella?"},
#          {"role": "user", "content": "yes pls"},
#         ]
msgs = [{"role": "system", "content":system_prompt} ,{"role": "user", "content": user_query}]

res = get_oai_response(user_query, "gpt-4-0125-preview", functions=functions, msgs=msgs)
print(res)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_NVPCmdRCtY9lO7sVhOFRPS6R', function=Function(arguments='{"location":"Boston, MA","unit":"f"}', name='getCurrentWeather'), type='function')]))


## Function.cpp

In [8]:
import openai


def get_mistral_rubra_response(prompt, model, functions, msgs):
  openai.api_key = "sk-"
  openai.base_url = "http://localhost:8019/v1/"
  
  try:
    completion = openai.chat.completions.create(
      model=model,
      temperature=0.1,
      messages=msgs,
      tools=functions,
      tool_choice="auto",
      # functions=functions,
      # function_call="auto",
      stream=False,
    )
    return completion.choices[0]
  except Exception as e:
    print(e, model, prompt)

system_prompt = "You are a helpful assistant."
functions = [
   {
      "type": "function",
    "function": {
      "name": "getCurrentWeather",
      "description": "Get the weather in location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
          "unit": {"type": "string", "enum": ["c", "f"]}
        },
        "required": ["location"]
      }
    }
  },
   {    "type": "function",
     "function":
    {
        "name": "orderUmbrella",
        "description": "Do this to help user to order an umbrella online", 
        "parameters": {
            "type": "object",
            "properties": {
                "brand_name": {
                    "type": "string",
                    "description": "The name of the umbrella brand"
                }
            },
            "required": [
                "brand_name"
            ]
        }
    }},
]


user_query = "check the weather in boston"
# msgs = [{"role": "system", "content":system_prompt} ,{"role": "user", "content": user_query}, {"role": "function", "content": '<<functions>>[getCurrentWeather(location="Boston)]'}, {"role": "observation", "content": "<<observation>>72 f, rainy."}
#         ,
#         {"role": "assistant", "content": "The current weather in Boston is 72 degrees Fahrenheit and it's raining. Would you like to order an umbrella?"},
#          {"role": "user", "content": "yes pls"},
#         ]
msgs = [{"role": "system", "content":system_prompt} ,{"role": "user", "content": user_query}]

res = get_mistral_rubra_response(user_query, "gpt-4-0125-preview", functions=functions, msgs=msgs)
print(res)

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id=0, function=[Function(arguments='{"location":"Boston","unit":"c"}', name='getCurrentWeather')], type='function')]))


In [7]:
system_prompt = "You are a helpful assistant."
functions = [
    {"function":
    {
        "name": "get_stock_fundermentals",
        "description": "Get the stock fundermentals data",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol, e.g. AAPL, GOOG"
                }
            },
            "required": [
                "symbol"
            ]
        }
    }},
    {"function":{
        "name": "check_word_anagram",
        "description": "Check if two words are anagrams of each other",
        "parameters": {
            "type": "object",
            "properties": {
                "word1": {
                    "type": "string",
                    "description": "The first word"
                },
                "word2": {
                    "type": "string",
                    "description": "The second word"
                }
            },
            "required": [
                "word1",
                "word2"
            ]
        }
    }}
]

user_query = "What's the stock fundementals of Tesla and google"

# 
# msgs = [{"role": "system", "content":system_prompt} ,{"role": "user", "content": user_query}, {"role": "function", "content": '<<functions>>[get_stock_price(symbol="TSLA")], <<functions>>[get_stock_price(symbol="GOOG")]'}, {"role": "observation", "content": "{'symbol': 'TSLA', 'company_name': 'Tesla, Inc.', 'sector': 'Consumer Cyclical', 'industry': 'Auto Manufacturers', 'market_cap': 611384164352, 'pe_ratio': 49.604652, 'pb_ratio': 9.762013, 'dividend_yield': None, 'eps': 4.3, 'beta': 2.427, '52_week_high': 299.29, '52_week_low': 152.37}}"}]
msgs = [{"role": "system", "content":system_prompt} ,{"role": "user", "content": user_query},]
res = get_mistral_rubra_response(user_query, "mistral_rubra", functions=functions, msgs=msgs)
print(res.message.content)

 <<functions>>[get_stock_fundermentals(symbol="TSLA")]
<<functions>>[get_stock_fundermentals(symbol="GOOG")]


In [19]:
import ast

input_str = "[get_current_weather(location='Boston, MA', api_key=123456789, unit='fahrenheit'), func('cde', x=1, b='2', c=[1, 2, {'a': 1, 'b': 2}])]"

# Parse the string into an AST
parsed_ast = ast.parse(input_str, mode='eval')

# Function to convert an AST node to a Python object
def ast_node_to_object(node):
    if isinstance(node, ast.Constant):
        return node.value
    elif isinstance(node, ast.List):
        return [ast_node_to_object(n) for n in node.elts]
    elif isinstance(node, ast.Dict):
        return {ast_node_to_object(key): ast_node_to_object(value) for key, value in zip(node.keys, node.values)}
    elif isinstance(node, ast.Tuple):
        return tuple(ast_node_to_object(n) for n in node.elts)
    # Add more cases here as needed
    return None

def find_calls(node):
    calls = []
    if isinstance(node, ast.Call):  # If it's a function call
        calls.append(node)
    for child in ast.iter_child_nodes(node):
        calls.extend(find_calls(child))
    return calls

# Extract all function call nodes
calls = find_calls(parsed_ast.body)

functions = []
for call in calls:
    if isinstance(call.func, ast.Name):  # Ensure it's a named function
        function_name = call.func.id
        args = [ast_node_to_object(arg) for arg in call.args]  # Convert all positional arguments
        kwargs = {kw.arg: ast_node_to_object(kw.value) for kw in call.keywords}  # Convert all keyword arguments
        functions.append((function_name, args, kwargs))

print(functions)


[('get_current_weather', [], {'location': 'Boston, MA', 'api_key': 123456789, 'unit': 'fahrenheit'}), ('func', ['cde'], {'x': 1, 'b': '2', 'c': [1, 2, {'a': 1, 'b': 2}]})]


In [20]:
import ast

input_str = "[get_current_weather(location='Boston, MA', api_key=123456789, unit='fahrenheit'), func('cde', x=1, b='2', c=[func_nested(1, 2), {'a': func_deep('value')}])]"

def ast_node_to_object(node):
    if isinstance(node, ast.Constant):
        return node.value
    elif isinstance(node, ast.List):
        return [ast_node_to_object(n) for n in node.elts]
    elif isinstance(node, ast.Dict):
        return {ast_node_to_object(key): ast_node_to_object(value) for key, value in zip(node.keys, node.values)}
    elif isinstance(node, ast.Tuple):
        return tuple(ast_node_to_object(n) for n in node.elts)
    elif isinstance(node, ast.Call):
        return ast.unparse(node)
        # Handle function calls: convert to a representation with the function name and arguments
        # func_name = ast_node_to_object(node.func)  # Get the function name
        # args = [ast_node_to_object(arg) for arg in node.args]  # Convert all positional arguments
        # kwargs = {kw.arg: ast_node_to_object(kw.value) for kw in node.keywords}  # Convert all keyword arguments
        # return {"function": func_name, "args": args, "kwargs": kwargs}
    elif isinstance(node, ast.Name):
        return node.id  # Return the identifier name
    # Add more cases here as needed
    return None

# Parse the string into an AST
parsed_ast = ast.parse(input_str, mode='eval')

# Function to find only the top-level Call nodes
def find_top_level_calls(node):
    calls = []
    if isinstance(node, ast.Call):  # If it's a function call
        calls.append(node)
        # Do not descend into child nodes to ensure we're only capturing top-level calls
        return calls
    for child in ast.iter_child_nodes(node):
        # Recursively find calls without going into nested calls
        calls.extend(find_top_level_calls(child))
    return calls

# Extract all top-level function call nodes
top_level_calls = find_top_level_calls(parsed_ast.body)

# Process each call node to get the details you want
functions = []
for call in top_level_calls:
    if isinstance(call.func, ast.Name):  # Ensure it's a named function
        function_name = call.func.id
        args = [ast_node_to_object(arg) for arg in call.args]  # Convert all positional arguments
        kwargs = {kw.arg: ast_node_to_object(kw.value) for kw in call.keywords}  # Convert all keyword arguments
        functions.append((function_name, args, kwargs))

print(functions)


[('get_current_weather', [], {'location': 'Boston, MA', 'api_key': 123456789, 'unit': 'fahrenheit'}), ('func', ['cde'], {'x': 1, 'b': '2', 'c': ['func_nested(1, 2)', {'a': "func_deep('value')"}]})]


46